## Recommender Models Based on Conditions

- Recommender Systems
    - Cosine similarity -- X + Y
- Modeling
    - OHE, cosine similarity, sort similarities within function
    - Orientation, age range, religion, offspring sentiment

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse
import sys
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from category_encoders import OneHotEncoder

In [2]:
# read in full cupid data
cupid = pd.read_pickle('data/clean_cupid.pkl')
cupid.drop(columns = ['status', 'location'], inplace = True)

In [3]:
cupid.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,22,m,straight,a little extra,strictly anything,socially,never,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,sometimes
1,35,m,straight,average,mostly other,often,sometimes,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,no
2,38,m,straight,thin,anything,socially,never,doesn't have kids,has cats,atheism,no


In [3]:
# read in grouped cupid data
#cupid_religion = pd.read_pickle('data/cupid_religion.pkl')

cupid_df = pd.read_pickle('data/grouped_cupid.pkl')

In [46]:
cupid_df.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,22,m,straight,average,anything,sometimes,no,"doesn't have kid(s), but wants kid(s)",likes dogs and cats,agnosticism,sometimes
1,35,m,straight,average,other,yes,sometimes,"doesn't have kid(s), but wants kid(s)",likes dogs and cats,doesn't matter,no
2,38,m,straight,thin,anything,sometimes,no,doesn't have kids,likes cats,doesn't matter,no


---

#### Religion

In [15]:
# function to ohe and return the cosine similarity based on orientation

def invalue_to_similarity(invalue_df, orientation_df):
    """
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # concat input values to orientation df to prep for cosine similarity
    df = pd.concat([orientation_df, invalue_df])

    # ohe
    df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)
    
    # make cosine_similarity input (input X)
    cosine_input = pd.DataFrame(df_encoded.iloc[-1]).T
    
    # drop last encoded row (input Y)
    df_encoded.drop(df_encoded.tail(1).index, inplace = True)
    
    # cosine_similarity
    similarity = cosine_similarity(cosine_input, df_encoded)
    
    # return top 5 matches
    top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]
    
    # return top 5 matches in a df with cosine similarities
    results = pd.DataFrame(columns = cupid.columns)

    for i in top5.index:
        results = results.append(pd.DataFrame(cupid.loc[i]).T)

    matches = pd.merge(top5, results, on = top5.index)
    matches.rename(columns = {'key_0' : 'user_id'}, inplace = True)
    matches.set_index('user_id', inplace = True)
    
    return matches

In [16]:
def lover_recommender_test6(invalue, religion, lowest_age, highest_age):
    """
    invalue (list): survey/streamlit app responses
    df = based on conditional -- if religion matters
    """
   
    # convert input to DataFrame
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})

    # ----------------
    
    # straight female looking for straight mmale
    if invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'f':
        
        # straight male
        straight_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(3000)
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity(invalue_df, straight_male)
    
    # straight male looking for straight female
    elif invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'm':
        
        # straight female
        straight_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(3000)

        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity(invalue_df, straight_female)
    
    # gay male looking for gay male
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'm':
        
        # gay male
        gay_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity(invalue_df, gay_male)
    
    # gay female looking for gay female
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'f':
        
        # gay female
        gay_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity(invalue_df, gay_female)
    
    # bisexual male/female looking for bisexual male/female
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f') or \
         (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):
        
        # bi individual
        bi = cupid_df[(cupid_df['orientation'] == 'bisexual') & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity(invalue_df, bi)

In [18]:
# input / up + down arrow

# age + age range
age = int(input('How old are you?'))
print("What's your age range?")
lowest_age = int(input('Lowest age?'))
highest_age = int(input('Highest age?'))

# dropdowns
sex = str(input('What gender do you identify as?'))
orientation = str(input('What sexual orientation do you identify as?'))
body_type = str(input("What's your body type?"))
diet = str(input('What does your diet consist of?'))
drinks = str(input('Do you consume alcoholic beverages?'))
drugs = str(input('Do you use drugs?'))
offspring = str(input('Do you have children and/or plan on having [more] children?'))
pets = str(input("What's your sentiment on dogs and/or cats"))
smokes = str(input("Do you smoke?"))

# religion
religion_matter = str(input('Does religion matter?'))
if religion_matter == 'yes':
    religion = str(input("What's your religion?"))
else:
    religion = "doesn't matter"

invalue = np.array([age, sex, orientation, body_type, diet, drinks, drugs, offspring, pets, religion, smokes])

How old are you? 36


What's your age range?


Lowest age? 30
Highest age? 45
What gender do you identify as? f
What sexual orientation do you identify as? gay
What's your body type? thin
What does your diet consist of? vegan
Do you consume alcoholic beverages? yes
Do you use drugs? yes
Do you have children and/or plan on having [more] children? doesn't have kids
What's your sentiment on dogs and/or cats likes dogs
Do you smoke? no
Does religion matter? yes
What's your religion? buddhism


In [19]:
lover_recommender_test6(invalue, religion, lowest_age, highest_age)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
21450,0.545455,31,f,gay,average,strictly vegan,socially,never,doesn't have kids,likes dogs and has cats,buddhism,no
52308,0.545455,36,f,gay,a little extra,anything,not at all,never,doesn't have kids,likes dogs,buddhism and somewhat serious about it,no
38756,0.545455,32,f,gay,average,mostly vegetarian,socially,never,doesn't have kids,has dogs,buddhism,no
58860,0.454545,31,f,gay,fit,strictly other,not at all,never,doesn't have kids,has cats,buddhism,no
6094,0.454545,31,f,gay,rather not say,anything,not at all,never,doesn't have kids,dislikes dogs and dislikes cats,buddhism,no


---

#### Add Offspring

In [55]:
cupid_df['offspring'].value_counts()

doesn't have kids                          41250
doesn't have kid(s), but wants kid(s)       7303
has kid(s)                                  3627
doesn't want kids                           2686
doesn't have kids, and doesn't want any     1080
has kid(s), but doesn't want more            702
has kid(s) and wants more                    425
wants kid(s)                                 400
Name: offspring, dtype: int64

In [4]:
# function to ohe, create sparse matrices, and return the cosine similarity based on orientation

def invalue_to_similarity2(invalue_df, orientation_df):
    """
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # split by offspring preference
    if invalue_df['offspring'].unique()[0] == "doesn't have kid(s), but wants kid(s)":
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "has kid(s), but doesn't want more")]
    elif invalue_df['offspring'].unique()[0] == "doesn't have kids, and doesn't want any":
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't want kids") | \
                                        (orientation_df['offspring'] == "doesn't have kids, and doesn't want any")]
    elif invalue_df['offspring'].unique()[0] == "has kid(s), but doesn't want more":
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't have kid(s), but wants kid(s)") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    elif invalue_df['offspring'].unique()[0] == "has kid(s) and wants more":
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    
    # concat input values to orientation df to prep for cosine similarity
    df = pd.concat([orientation_df, invalue_df])

    # ohe
    df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)
    
    # make cosine_similarity input (input X)
    cosine_input = pd.DataFrame(df_encoded.iloc[-1]).T
    
    # drop last encoded row (input Y -- data for input X to reference)
    df_encoded.drop(df_encoded.tail(1).index, inplace = True)
    
    # cosine_similarity(X, y)
    similarity = cosine_similarity(cosine_input, df_encoded)
    
    # return top 5 matches
    top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]
    
    # return top 5 matches in a df with cosine similarities
    results = pd.DataFrame(columns = cupid.columns)

    for i in top5.index:
        results = results.append(pd.DataFrame(cupid.loc[i]).T)

    matches = pd.merge(top5, results, on = top5.index)
    matches.rename(columns = {'key_0' : 'user_id'}, inplace = True)
    matches.set_index('user_id', inplace = True)
    
    return matches

In [22]:
invalue = ['28', 'm', 'straight', 'fit', 'anything', 'yes', 'no', "doesn't have kids, and doesn't want any", 'likes cats', 'christianity', "no"]
# highest_age = 24
# lowest_age = 43

invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})
invalue_df

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,28,m,straight,fit,anything,yes,no,"doesn't have kids, and doesn't want any",likes cats,christianity,no


In [5]:
def lover_recommender_test7(invalue, religion, lowest_age, highest_age):
    """
    invalue (list): survey/streamlit app responses
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
   
    # convert input from array to DataFrame
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})

    # ----------------
    
    # straight female looking for straight mmale
    if invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'f':
        
        # straight male df
        straight_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(10000)
        
        # call 'invalue_to_similarity' function to return top 5 similarities
        return invalue_to_similarity2(invalue_df, straight_male)
    
    # straight male looking for straight female
    elif invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'm':
        
        # straight female df
        straight_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(10000)

        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity2(invalue_df, straight_female)
    
    # gay male looking for gay male
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'm':
        
        # gay male df
        gay_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity2(invalue_df, gay_male)
    
    # gay female looking for gay female
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'f':
        
        # gay female df
        gay_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity2(invalue_df, gay_female)
    
    # bisexual male/female looking for bisexual male/female
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f') or \
         (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):
        
        # bi individual df
        bi = cupid_df[(cupid_df['orientation'] == 'bisexual') & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        # call 'invalue_to_similarity' function to return similarities
        return invalue_to_similarity2(invalue_df, bi)

In [20]:
# input / up + down arrow

# age + age range
age = int(input('How old are you?'))
print("What's your age range?")
lowest_age = int(input('Lowest age?'))
highest_age = int(input('Highest age?'))

# dropdowns
sex = str(input('What gender do you identify as?'))
orientation = str(input('What sexual orientation do you identify with?'))
body_type = str(input("What's your body type?"))
diet = str(input('What does your diet consist of?'))
drinks = str(input('Do you consume alcoholic beverages?'))
drugs = str(input('Do you use drugs?'))

# offspring
# do you have any children? yes/no
any_children = str(input('Do you have any children?'))

# do you want any [more] children? yes/no
more_children = str(input('Do you want any [more] children?'))

if any_children == 'no' and more_children == 'yes':
    offspring = "doesn't have kid(s), but wants kid(s)"
elif any_children == 'no' and more_children == 'no':
    offspring = "doesn't have kids, and doesn't want any"
elif any_children == 'yes' and more_children == 'no':
    offspring = "has kid(s), but doesn't want more"
elif any_children == 'yes' and more_children == 'yes':
    offspring = "has kid(s) and wants more"

pets = str(input("What's your sentiment on dogs and/or cats"))
smokes = str(input("Do you smoke?"))

# religion
religion_matter = str(input('Does religion matter?'))
if religion_matter == 'yes':
    religion = str(input("What's your religion?"))
else:
    religion = "doesn't matter"

invalue = np.array([age, sex, orientation, body_type, diet, drinks, drugs, offspring, pets, religion, smokes])

How old are you? 28


What's your age range?


Lowest age? 24
Highest age? 43
What gender do you identify as? m
What sexual orientation do you identify with? straight
What's your body type? fit
What does your diet consist of? anything
Do you consume alcoholic beverages? yes
Do you use drugs? no
Do you have any children? no
Do you want any [more] children? no
What's your sentiment on dogs and/or cats likes cats
Do you smoke? christianity
Does religion matter? no


In [9]:
# invalue = ['28', 'm', 'straight', 'fit', 'anything', 'yes', 'no', "doesn't have kids, and doesn't want any", 'likes cats', 'christianity', 'no']
# highest_age = 24
# lowest_age = 43
# religion = 'christianity'

lover_recommender_test7(invalue, religion, lowest_age, highest_age)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
1857,0.727273,26,f,straight,athletic,anything,often,never,doesn't have kids,has cats,christianity,no
25124,0.636364,27,f,straight,average,mostly anything,rarely,never,"doesn't have kids, and doesn't want any",has cats,christianity,no
41612,0.636364,34,f,straight,rather not say,anything,often,never,doesn't have kids,likes cats,christianity,no
58045,0.636364,24,f,straight,athletic,mostly anything,often,never,doesn't have kids,has dogs,christianity and somewhat serious about it,no
10855,0.636364,24,f,straight,fit,anything,desperately,never,doesn't have kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,no


In [12]:
# invalue = ['36', 'm', 'bisexual', 'fit', 'vegan', 'yes', 'sometimes', "doesn't have kid(s), but wants kid(s)", 'likes dogs and cats', 'christianity', 'yes']
# highest_age = 30
# lowest_age = 45
# religion = 'christianity'

lover_recommender_test7(invalue, religion, lowest_age, highest_age)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
432,0.363636,31,m,bisexual,used up,anything,very often,often,has kids,has dogs and dislikes cats,christianity and very serious about it,no
25664,0.363636,35,f,bisexual,curvy,mostly anything,often,never,"has kids, and might want more",has dogs,christianity,yes
42459,0.363636,43,f,bisexual,athletic,mostly anything,socially,never,"has kids, but doesn't want more",likes dogs and has cats,christianity and somewhat serious about it,no
24990,0.181818,32,f,bisexual,average,anything,rarely,never,has kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,sometimes
